# Random forest fitting
This script is inspired on the 02-experiment-tracking HW, where you fit a bunch of random-forest models to the taxi data, and log onto MLflow both the fitted model as well as the dictionary vectorizer.

## Starting up MLflow
To start up mlflow, run on bash in this folder the following:
```bash
mlflow server --backend-store-uri=sqlite:///mlflow.db --default-artifact-root=./artifacts_local/
```

Note that the default artifact root can be redone with an AWS bucket (s3) and that the backend store uri can just be a setup postgres

In [1]:
import pickle
import pandas as pd
import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

/Users/andreanicolas/.local/share/virtualenvs/web-service-mlflow-LwFdWyP1/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Extra: setting up a pipeline

In [2]:
from sklearn.pipeline import make_pipeline

## Setting up MLflow on the script

In [3]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('green-taxi-duration-andy')

<Experiment: artifact_location=('/Users/andreanicolas/Library/CloudStorage/GoogleDrive-andreanicolas91@gmail.com/My '
 'Drive/ASU_ComputerScience/MLOps_studies/04-deployment/web-service-mlflow/artifacts_local/1'), creation_time=1717570527095, experiment_id='1', last_update_time=1717570527095, lifecycle_stage='active', name='green-taxi-duration-andy', tags={}>

## Setting up functions

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

## Reading in the data and prepping dicts

In [5]:
df_train = read_dataframe('../../data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('../../data/green_tripdata_2023-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

## Logging things on MLflow

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    # extra: use a pipeline instead of independent dv/models

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred,y_val)
    print(params,rmse)
    mlflow.log_metric('rmse',rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path='model')

    # old code:
    # dv = DictVectorizer()
    # model = RandomForestRegressor(**params, n_jobs=-1)

    # X_train = dv.fit_transform(dict_train)
    # model.fit(X_train,y_train)

    # X_val = dv.transform(dict_val)
    # y_pred = model.predict(X_val)

    # rmse = root_mean_squared_error(y_pred,y_val)
    # print(params,rmse)
    # mlflow.log_metric('rmse',rmse)

    # mlflow.sklearn.log_model(model, artifact_path='model')

    # with open('dict_vectorizer.bin','wb') as f_out:
    #     pickle.dump(dv,f_out)

    # mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 5.39920274232368
